In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "question": "What do you know about Avatar?",
        "answer": """
        Here is what I know:
        Director: James Cameron
        Main Cast: Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang
        Box Office Revenue: $2.92 billion
        Genre: Science Fiction, Adventure, Action
        Synopsis: Set in the mid-22nd century, Avatar follows Jake Sully, a paraplegic former Marine who becomes involved with the indigenous Na'vi people on the distant planet Pandora. Tasked with infiltrating the Na'vi to aid human mining efforts, Jake soon finds himself torn between following orders and protecting the planet and its inhabitants.
        """,
    },
    {
        "question": "What do you know about Titanic?",
        "answer": """
        I know this:
        Director: James Cameron
        Main Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates
        Box Office Revenue: $2.25 billion
        Genre: Romance, Drama, Historical
        Synopsis: Set against the backdrop of the ill-fated maiden voyage of the RMS Titanic, the film tells the story of Jack and Rose, two people from different social classes who fall in love. As the ship meets its tragic end, their love story becomes a tale of passion and sacrifice.
        """,
    },
    {
        "question": "What do you know about The Dark Knight?",
        "answer": """
        I know this:
        Director: Christopher Nolan
        Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine, Gary Oldman
        Box Office Revenue: $1.006 billion
        Genre: Action, Crime, Drama
        Synopsis: In The Dark Knight, Batman faces his greatest challenge yet as he battles the anarchist mastermind known as the Joker. As chaos engulfs Gotham City, the film explores themes of heroism, moral ambiguity, and the limits of justice, with Heath Ledger's iconic performance as the Joker earning critical acclaim.
        """,
    },
]

: 

In [5]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [8]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {movie}?",
    input_variables=["movie"],
)



Human: What do you know about Avatar?
AI:
        Here is what I know:
        Director: James Cameron
        Main Cast: Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang
        Box Office Revenue: $2.92 billion
        Genre: Science Fiction, Adventure, Action
        Synopsis: Set in the mid-22nd century, Avatar follows Jake Sully, a paraplegic former Marine who becomes involved with the indigenous Na'vi people on the distant planet Pandora. Tasked with infiltrating the Na'vi to aid human mining efforts, Jake soon finds himself torn between following orders and protecting the planet and its inhabitants.
        

Human: What do you know about Inception?


In [11]:
chain = prompt | chat


response = chain.invoke({"movie": "Mr. Nobody"})
print(response)

AI:
        I know this:
        Director: Jaco Van Dormael
        Main Cast: Jared Leto, Sarah Polley, Diane Kruger, Linh Dan Pham
        Box Office Revenue: $1.4 million
        Genre: Science Fiction, Drama, Romance
        Synopsis: The film follows the life of Nemo Nobody, the last mortal on Earth in the year 2092. As he reflects on his past and the different choices he could have made, the film explores themes of love, destiny, and the nature of existence.content='AI:\n        I know this:\n        Director: Jaco Van Dormael\n        Main Cast: Jared Leto, Sarah Polley, Diane Kruger, Linh Dan Pham\n        Box Office Revenue: $1.4 million\n        Genre: Science Fiction, Drama, Romance\n        Synopsis: The film follows the life of Nemo Nobody, the last mortal on Earth in the year 2092. As he reflects on his past and the different choices he could have made, the film explores themes of love, destiny, and the nature of existence.'
